In [ ]:
from google.colab import drive

drive.mount('/content/drive/', force_remount=True)
%cd '/content/drive/Shared drives/MinneMUDAC/Data Challenge details'


Mounted at /content/drive/
/content/drive/Shared drives/MinneMUDAC/Data Challenge details


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from pandas import DataFrame
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.stats import reciprocal
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows',300)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pickle
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.inspection import PartialDependenceDisplay,partial_dependence

In [ ]:
df_teambyteam = pd.read_csv('/content/drive/Shared drives/MinneMUDAC/Model/pdp/df.csv')
mapping_df_all = pd.read_csv('/content/drive/Shared drives/MinneMUDAC/Model/pdp/df_mapping.csv')

In [ ]:
#df_teambyteam = df_teambyteam[df_teambyteam['HomeTeam'].isin(range(0,6))] # Lan
# df_teambyteam = df_teambyteam[df_teambyteam['HomeTeam'].isin(range(6,12))] # Simin
# df_teambyteam = df_teambyteam[df_teambyteam['HomeTeam'].isin(range(12,18))] # Rio
# df_teambyteam = df_teambyteam[df_teambyteam['HomeTeam'].isin(range(18,24))] # Congyi
df_teambyteam = df_teambyteam[df_teambyteam['HomeTeam'].isin([24, 25, 27, 28, 29])] # Jichen

In [ ]:
Home_Team = df_teambyteam.HomeTeam.unique().tolist()

In [ ]:
len(Home_Team)

6

In [ ]:
def custom_split(X,  groups):
  for train_index, test_index in groups:
    original_train_index = np.array(X[X['date_int'].isin(train_index+1)].index)
    original_test_index = np.array(X[X['date_int'].isin(test_index+1)].index)
    yield original_train_index, original_test_index

In [ ]:
rmse_all_team_test = []
rmse_team_by_team_train = []
feature_important = []

from sklearn.metrics import make_scorer, mean_absolute_percentage_error
scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

# pdp_team_all = pd.DataFrame({'feature_values': [], 'partial_dependence_values': [],"feature":[], "team":[]})

for team in Home_Team:
  team_model = df_teambyteam[df_teambyteam['HomeTeam'] == team]
  train_df = team_model[team_model['Date']<20220101]
  test_df = team_model[team_model['Date']>20211231]

  train_df['date_int']=pd.to_datetime(train_df['Date']).rank(method="dense").apply(int)
  train_df = train_df.reset_index()
  train_df = train_df.drop(columns='index')

  test_df['date_int']=pd.to_datetime(test_df['Date']).rank(method="dense").apply(int)
  test_df = test_df.reset_index()
  test_df = test_df.drop(columns='index')

  X_train = train_df.drop(columns='Attendance')
  y_train = train_df['Attendance']
  X_train = train_df.drop(columns='Attendance')
  X_test = test_df.drop(columns=['Attendance'])
  y_test = test_df['Attendance']

  tscv = TimeSeriesSplit(n_splits=20, test_size=50)
  groups = tscv.split(X_train['date_int'].unique())
  lgb = LGBMRegressor()
  param_grid = {
    'task':['train'],
    'boosting': ['gbdt'],
    # 'objective' : ['root_mean_squared_error'],
    # 'metric':['rmse'],
    'objective' : ['mape'],
    'metric':['mape'],
    'learning_rate':  reciprocal(1e-3, 3e-1),
    'max_depth' :list(range(5,30)) ,
    'num_leaves':list(range(5,300,5)),
    'min_data_in_leaf':list(range(10,50,5)),
    'bagging_fraction': [ 0.4, 0.6, 0.8 ],
    'bagging_freq': [   100, 200, 400  ],
    'n_estimators': list(range(100,500,50)),
    'subsample': [0.5],
    'lambda_l2': list(range(0,7)),
    'max_depth': list(range(10,20))
    }
  grid = RandomizedSearchCV(lgb,
                            n_iter = 60,
                            # scoring = 'neg_root_mean_squared_error',
                            scoring=scorer,
                            param_distributions = param_grid,
                            cv=custom_split(X_train, groups))
  grid.fit(X_train,y_train,verbose=False)

  model = LGBMRegressor(subsample = grid.best_params_['subsample'],
                      objective = grid.best_params_['objective'],
                      num_leaves = grid.best_params_['num_leaves'],
                      n_estimators=grid.best_params_['n_estimators'],
                      min_data_in_leaf=grid.best_params_['min_data_in_leaf'],
                      metric= grid.best_params_['metric'],
                      max_depth= grid.best_params_['max_depth'],
                      learning_rate= grid.best_params_['learning_rate'],
                      lambda_l2= grid.best_params_['lambda_l2'],
                      boosting= grid.best_params_['boosting'],
                      bagging_freq= grid.best_params_['bagging_freq'],
                      bagging_fraction= grid.best_params_['bagging_fraction'])
  model.fit(X_train, y_train,verbose=False)


  team_name = mapping_df_all[(mapping_df_all["feature"]=="HomeTeam") & (mapping_df_all["encoded_value"]==team)]["original_category"].values[0]



  rmse_team_by_team_train = np.append(rmse_team_by_team_train,grid.best_score_)
  # gbm_team_by_team = grid.best_estimator_
  gbm_team_by_team = model
  importances = gbm_team_by_team.feature_importances_
  feature_important = np.append(feature_important,importances)
  feature_imp = pd.DataFrame(sorted(zip(importances,X_train.columns)), columns=['Value','Feature']).sort_values("Value", ascending = False)
  feature_imp.to_csv('/content/drive/Shareddrives/MinneMUDAC/Model/retrain_feature_importance_'+team_name+'30.csv',index=False)


  pdp_team = pd.DataFrame({'feature_values': [], 'partial_dependence_values': [],"feature":[]})
  important_feature = feature_imp["Feature"][:16].tolist()
  for feature in important_feature:
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.set_title("Decision Tree")
    tree_disp = PartialDependenceDisplay.from_estimator(model, X_train, features = [feature], ax=ax)
    tree_disp.figure_.savefig('/content/drive/Shared drives/MinneMUDAC/Model/pdp/'+team_name+"_"+feature+'.png', dpi=300, bbox_inches='tight')


    pdp = partial_dependence(model, X_train, features = [feature])
    partial_dependence_values, feature_values = pdp["average"][0].tolist(), pdp["values"][0].tolist()
    pdp_df = pd.DataFrame({'feature_values': feature_values,'partial_dependence_values': partial_dependence_values})
    pdp_df["feature"] = feature
    pdp_team = pd.concat([pdp_team, pdp_df])


  pdp_team["team"] = team_name
  pdp_team.to_csv('/content/drive/Shared drives/MinneMUDAC/Model/pdp/pdp_data_'+team_name+'.csv',index=False)
  #pdp_team_all = pd.concat([pdp_team_all, pdp_team])

  model_path = '/content/drive/Shareddrives/MinneMUDAC/Model/retrain_lgbm_model_'+team_name+'30.bin'
  pickle.dump(gbm_team_by_team, open(model_path, 'wb'))

# pdp_team_all.to_csv('/content/drive/Shared drives/MinneMUDAC/Model/pdp/pdp_data_all.csv',index=False)